## Point design analysis

In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from rocketcea.cea_obj_w_units import CEA_Obj
from rocketcea.cea_obj import add_new_fuel

### Global definitions

In [2]:
PSI_TO_PA = 6894.7572931783
M_TO_IN = 39.3701

# define PMMA
card_str = '''
fuel PMMA C 5 H 8 O 2
h,kj=-430.5 t(k)=299.82
''' # Greg Zilliac's recommendation for modeling PMMA
add_new_fuel('PMMA', card_str) # rocketCEA function to add PMMA to possible inputs

### Functions

In [41]:
def get_CEA_results(Pc, Pe, OF):
    C = CEA_Obj(oxName='GOX', fuelName='PMMA',
        isp_units='sec',
        cstar_units='m/s',
        sonic_velocity_units='m/s')

    PcOvPe = Pc/Pe
    AR = C.get_eps_at_PcOvPe(Pc=Pc, MR=OF, PcOvPe=PcOvPe)
    _, gam_chamber = C.get_Chamber_MolWt_gamma(Pc=Pc, MR=OF, eps=AR)
    ISP = C.get_Isp(Pc=Pc, MR=OF, eps=AR)
    Cstar = C.get_Cstar(Pc=Pc, MR=OF)
    _, at, _ = C.get_SonicVelocities(Pc=Pc, MR=OF, eps=AR)

    return AR, gam_chamber, ISP, Cstar, at

def get_exit_pressure(Pc, gamma, AR):
    '''
    Find exit pressure from Pc, gam and AR.
    '''
    def f(x):
        temp1 = ((gamma + 1)/2) ** (1/(gamma -1))
        temp2 = (x/Pc) ** (1/gamma)
        temp3 = (gamma+1) / (gamma-1)
        temp4 = 1 - (x/Pc)**((gamma-1)/gamma);
        return temp1*temp2*np.sqrt(temp3*temp4) - 1/AR
    guess = 101325.
    Pe = fsolve(f, guess, xtol=1e-10)
    return Pe[0]

def get_thrust_coeff(Pc, Pe, P0, gamma, AR):
    temp1 = (2*gamma**2) / (gamma-1)
    temp2 = (2/(gamma+1)) ** ((gamma+1)/(gamma-1))
    temp3 = 1 - (Pe/Pc) ** ((gamma-1)/gamma)
    temp4 = (Pe-P0)/Pc * AR
    return np.sqrt(temp1 * temp2 * temp3) + temp4

### Point design inputs

In [42]:
Pc = 150.0 * PSI_TO_PA # chamber pressure
Pe = 101325.0 # exit pressure
P0 = 101325.0 # ambient pressure
OF = 1.287 # OF ratio
a = 8.96e-5 # regression rate
n = 0.35 # regression rate
rho_fuel = 1180 # fuel density

eff_nozzle = 0.992 # nozzle efficiency
eff_Cstar = 0.85 # Cstar efficiency

thrust = 60 # design point thrust
burn_time = 20 # burn time
R_outer = 3./2. / M_TO_IN # fuel grain outer radius
L = 18 / M_TO_IN # fuel grain length

In [ ]:
# use this cell to change values from default

thrust = 100
burn_time = 15
R_outer = 4./2. / M_TO_IN # fuel grain outer radius

### CEA solve

In [43]:
AR, gam_chamber, ISP, Cstar, at = get_CEA_results(Pc, Pe, OF)
print('AR                 = %f' % AR)
print('gam in the chamber = %f' % gam_chamber)
print('ISP                = %f [s]' % ISP)
print('Cstar              = %f [m/s]' % Cstar)
print('SoS at throat      = %f [m/s]' % at)

AR                 = 2.294931
gam in the chamber = 1.177445
ISP                = 262.208031 [s]
Cstar              = 1717.825601 [m/s]
SoS at throat      = 1155.807155 [m/s]


### Consistency check
Use ideal (isentropic/adiabatic) nozzle equations to check that the area ratio and $\gamma$ are compatible with a fully expanded nozzle at atmospheric pressure given the prescribed chamber pressure.

In [15]:
Pe_solved = get_exit_pressure(Pc, gam_chamber, AR)
print('Solved exit pressure using ideal nozzle = %f [Pa]' % Pe_solved)
rel_diff = (Pe_solved-Pe)/Pe
assert rel_diff < 0.03, 'Error the relative difference was %f' % rel_diff

Solved exit pressure using ideal nozzle = 104301.146808 [Pa]


### Get thrust coefficient and throat area
The imposed thrust $\dot{m}u_e + (p_e-p_0)A_e$ indirectly imposes some constraints on the absolute value of $A_e$ thus $A_t$. This allows us to compute the throat area for the required thrust.

In [19]:
CF = get_thrust_coeff(Pc, Pe_solved, P0, gam_chamber, AR)
At = thrust / (eff_nozzle * CF * Pc)
Rt = (At / np.pi) ** (1/2)
print('CF = %f' % CF)
print('At = %f [m2]' % At)
print('Rt = %f [m]' % Rt)

CF = 1.275216
At = 0.000046 [m2]
Rt = 0.003821 [m]


### Get mass flows
These are mass flows based on the required thrust.

In [21]:
mdot = Pc*At / (eff_Cstar*Cstar)
mdot_fuel = mdot / (OF+1)
mdot_ox = OF * mdot_fuel
print('mdot      = %f [kg/m3]' % mdot)
print('mdot_ox   = %f [kg/m3]' % mdot_ox)
print('mdot_fuel = %f [kg/m3]' % mdot_fuel)

mdot      = 0.032483 [kg/m3]
mdot_ox   = 0.018280 [kg/m3]
mdot_fuel = 0.014203 [kg/m3]


### Fuel grain dimensions
To achieve the previously obtained fuel mass flow, the burn surface needs to be sufficiently large. On the other hand, based on the burn time assuming constant thrust, there is a maximum allowable inner radius. 
Here we first compute $R_{i, \mathrm{max}}^{t=0}$. Then, given our desired fuel grain length, we compute a candidate value of $R_{i}^{t=0}$ that would satisfy the desired thrust. Of course we need to have $R_{i}^{t=0} < R_{i, \mathrm{max}}^{t=0}$.

In [30]:
R_inner_max = ( \
    R_outer**(2*n+1) - a*(2*n+1)*(mdot_ox/np.pi)**n * burn_time \
    ) ** (1/(2*n+1))
R_inner = ( \
    mdot_f/(a*(mdot_ox/np.pi)**n) * 1./(rho_fuel*2*np.pi*L) \
    ) ** (1/(1-2*n))
assert R_inner < R_inner_max, 'No valid inner radius satisfying burn time, thrust and grain length.'
R_after_burn = ( \
    burn_time*a*(2*n+1)*(mdot_ox/np.pi)**n + R_inner**(2*n+1) \
    ) ** (1/(2*n+1))
thickness_after_burn = R_after_burn - R_inner

print('Fuel inner radius   = %f [m] or %f [in]' % (R_inner, R_inner*M_TO_IN) )
print('Fuel outer radius   = %f [m] or %f [in]' % (R_outer, R_outer*M_TO_IN) )
print('Fuel inner diameter = %f [m] or %f [in]' % (R_inner*2, R_inner*2*M_TO_IN) )
print('R_inner_max         = %f [in]' % (R_inner_max*M_TO_IN))
print('Final thickness     = %f [m]' % thickness_after_burn)

Fuel inner radius   = 0.014931 [m] or 0.587844 [in]
Fuel outer radius   = 0.038100 [m] or 1.500000 [in]
Fuel inner diameter = 0.029862 [m] or 1.175688 [in]
R_inner_max         = 1.382032 [in]
Final thickness     = 0.005036 [m]


### Get mass flow for given thrust
For this, we first use thrust equation:
$$ T = \dot{m}u_e + (p_e-p_0)A_e $$
We impose $\mathrm{OF}$, $p_c$, $\gamma$, $\varepsilon$ and $A^*$. This imposes $A_e$ and $p_e=f(p_c, \gamma, \varepsilon)$.
In order to get $\dot{m}$, we thus need $u_e$. This is obtained using
$$ u_e = \frac{u_e}{u^*}u^* = \frac{a_e}{a^*}M_ea^* $$
$$ \frac{a_e}{a^*} = \left( \frac{p_e}{p^*} \right) ^ \frac{\gamma-1}{2\gamma} $$
$$ \frac{p_c}{p^*} = \left( \frac{\gamma+1}{2} \right) ^ \frac{\gamma-1}{\gamma} $$
$$ M_e = \sqrt{\frac{2}{\gamma-1} \left( \left( \frac{p_c}{p_e} \right) ^ \frac{\gamma-1}{\gamma} - 1 \right)} $$

In [48]:
T = 100
Mach_exit = np.sqrt( \
    2/(gam_chamber-1) * ( (Pc/Pe)**( (gam_chamber-1)/gam_chamber ) - 1) )
tmp = (Pe/Pc)**( (gam_chamber-1)/(2*gam_chamber) ) * \
    ((gam_chamber+1)/2)**(1/2)
u_exit = tmp * Mach_exit * at

mdot = (T - (Pe-P0)*AR*At) / u_exit
mdot_ox = OF/(OF+1) * mdot
print(mdot_ox)

0.02557385848868988


In [36]:
Mach_exit

2.173671777766226